# Interaction Matrix for Perspective, Parabolic and Hyperbolic

This document has the derivations of interaction matrices for the perspective, parabolic and hyperbolic projections. The interaction matrices are used to calculate the kinematics of the points in the image plane given camera inputs.

The following references were used for generating the models:

1. "Geometric Properties of Central Catadioptric Line Images and Their Application in Calibration",
by João P. Barreto and Helder Araujo, in IEEE Transactions on Pattern Analysis and Machine Intelligence,  VOL. 27, NO. 8, AUGUST 2005
2. "Epipolar Geometry of Central Projection Systems Using Veronese Maps" by João P. Barreto and Kostas Daniilidis,
in 2006 IEEE Computer Society Conference on Computer Vision and Pattern Recognition (CVPR'06)

This reference is also relevant:

1. "Photometric visual servoing for omnidirectional cameras", Guillaume Caron, Eric Marchand, El Mustapha Mouaddib, in Autonomous Robots 2013

The division model is seen in:

1. 4.2 of "Simultaneous linear estimation of multiple view geometry and lens distortion", Andrew W Fitzgibbon

In [1]:
import sympy as sp

sp.init_printing()

In [2]:
MODEL = "generalized"  # Choose between perspective, generalized, ...
GENERALIZED = "barreto"  # Choose between barreto or caron
USE_CALIBRATED_HC = True  # Use calibrated Hc matrix or not

We start by defining the varibles, then the imaging model, and finally we calculate the image-based visual servoing model for these cameras

In [3]:
# Define world points and time
t = sp.symbols("t")
X = sp.Function("X")
Y = sp.Function("Y")
Z = sp.Function("Z")

# Define camera properties
csi = sp.symbols("xi")
psi = sp.symbols("psi") 

# Auxiliary variables
alpha = sp.symbols("alpha")
rho = sp.sqrt(X(t) ** 2 + Y(t) ** 2 + Z(t) ** 2)
x = X(t)/(Z(t) + csi * rho)
y = -Y(t)/(Z(t) + csi * rho)
alp = sp.sqrt( 1 + (1 - csi**2) * (x ** 2 + y ** 2))
Mc = sp.Matrix.diag([psi - csi, csi - psi, 1])
# Mc = sp.Matrix.diag([-csi, csi, 1])
# Mc = sp.Matrix.diag([1, 1, 1])

# Define camera matrix

# Rc = sp.Matrix.diag([r11, 1, r33])
# Rc = sp.Matrix.eye(3)
Rc = sp.Matrix.diag([1, -1, -1])
Rc = sp.Matrix.diag([1, 1, 1])
Hc = Rc @ Mc

# Implement ground truth expression from Barreto's paper and subtract it to my solution: result needs to be zero
l11 = (1 + x**2 * (1 - csi *(alp + csi)) + y**2)/(rho*(alp + csi))
l12 = x * y * csi / rho
l13 = -x*alp/rho
l14 = x*y
l15 = ((1 + x**2)*alp - y**2 * csi)/(alp + csi)
l16 = y

l21 = -x*y*csi/rho
l22 = (1 +  x**2 + y**2 * (1 - csi *(alp + csi)))/(rho*(alp + csi))
l23 = -y*alp/rho
l24 = ((1 + y**2)*alp - x**2 * csi)/(alp + csi)
l25 = x * y
l26 = -x

barreto_L = sp.Matrix([[l11, l12, l13, l14, l15, l16],
                       [l21, l22, l23, l24, l25, l26]])

# barreto_L = sp.simplify(barreto_L)


# Imaging symbols
x = sp.symbols("x")
y = sp.symbols("y")

### Define image model

In [4]:
if MODEL == "generalized" and GENERALIZED == "caron":
    r = sp.symbols("rho")
    
    # Equation 3 in Caron et al
    Xp = X(t) / (Z(t) + csi * rho)
    Yp = Y(t) / (Z(t) + csi * rho)
    Zp = (Z(t) + csi * rho) / (Z(t) + csi * rho)
    img = sp.Matrix([[Xp, Yp, Zp]]).T

elif MODEL == "generalized" and GENERALIZED == "barreto":
    r = sp.symbols("rho")

    Xp = X(t) / rho
    Yp = -Y(t) / rho
    Zp = Z(t) / rho + csi
    img = sp.Matrix([[Xp/Zp, Yp/Zp, Zp/Zp]]).T

elif MODEL == "perspective":
    Xs = X(t) / rho
    Ys = Y(t) / rho
    Zs = Z(t) / rho

    Xp = Xs / Zs
    Yp = Ys / Zs
    Zp = Zs / Zs
    img = sp.Matrix([[Xp, Yp, Zp]]).T

    img = Hc @ img
else:
    raise SystemExit("Model unknown.")

img

⎡                            X(t)                           ⎤
⎢───────────────────────────────────────────────────────────⎥
⎢                                    _______________________⎥
⎢⎛               Z(t)           ⎞   ╱  2       2       2    ⎥
⎢⎜ξ + ──────────────────────────⎟⋅╲╱  X (t) + Y (t) + Z (t) ⎥
⎢⎜       _______________________⎟                           ⎥
⎢⎜      ╱  2       2       2    ⎟                           ⎥
⎢⎝    ╲╱  X (t) + Y (t) + Z (t) ⎠                           ⎥
⎢                                                           ⎥
⎢                           -Y(t)                           ⎥
⎢───────────────────────────────────────────────────────────⎥
⎢                                    _______________________⎥
⎢⎛               Z(t)           ⎞   ╱  2       2       2    ⎥
⎢⎜ξ + ──────────────────────────⎟⋅╲╱  X (t) + Y (t) + Z (t) ⎥
⎢⎜       _______________________⎟                           ⎥
⎢⎜      ╱  2       2       2    ⎟                           ⎥
⎢⎝    ╲╱

In [5]:
# Include Hc matrix
if USE_CALIBRATED_HC and MODEL == "generalized":
    # Hc = K @ Rc @ Mc
    img = Hc @ img

img

⎡                        (ψ - ξ)⋅X(t)                       ⎤
⎢───────────────────────────────────────────────────────────⎥
⎢                                    _______________________⎥
⎢⎛               Z(t)           ⎞   ╱  2       2       2    ⎥
⎢⎜ξ + ──────────────────────────⎟⋅╲╱  X (t) + Y (t) + Z (t) ⎥
⎢⎜       _______________________⎟                           ⎥
⎢⎜      ╱  2       2       2    ⎟                           ⎥
⎢⎝    ╲╱  X (t) + Y (t) + Z (t) ⎠                           ⎥
⎢                                                           ⎥
⎢                      -(-ψ + ξ)⋅Y(t)                       ⎥
⎢───────────────────────────────────────────────────────────⎥
⎢                                    _______________________⎥
⎢⎛               Z(t)           ⎞   ╱  2       2       2    ⎥
⎢⎜ξ + ──────────────────────────⎟⋅╲╱  X (t) + Y (t) + Z (t) ⎥
⎢⎜       _______________________⎟                           ⎥
⎢⎜      ╱  2       2       2    ⎟                           ⎥
⎢⎝    ╲╱

In [6]:
# Take time derivatives
sdimg = sp.diff(img, t)
sdimg

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢          ⎛       d               d               d       ⎞                  
⎢  (ψ - ξ)⋅⎜- X(t)⋅──(X(t)) - Y(t)⋅──(Y(t)) - Z(t)⋅──(Z(t))⎟⋅X(t)             
⎢          ⎝       dt              dt              dt      ⎠                  
⎢  ────────────────────────────────────────────────────────────── + ──────────
⎢                                                           3/2               
⎢   ⎛               Z(t)           ⎞ ⎛ 2       2       2   ⎞        ⎛         
⎢   ⎜ξ + ──────────────────────────⎟⋅⎝X (t) + Y (t) + Z (t)⎠        ⎜ξ + ─────
⎢   ⎜       _______________________⎟                                ⎜       __
⎢   ⎜      ╱  2       2       2    ⎟                

In [7]:
sdimg = sp.simplify(sdimg)
sdimg

⎡        ⎛  ⎛     _______________________       ⎞                             
⎢        ⎜  ⎜    ╱  2       2       2           ⎟ ⎛     d               d     
⎢(ψ - ξ)⋅⎜- ⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅⎜X(t)⋅──(X(t)) + Y(t)⋅──(Y(t
⎢        ⎝                                        ⎝     dt              dt    
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢        ⎛  ⎛     _______________________       ⎞                             
⎢        ⎜  ⎜    ╱  2       2       2           ⎟ ⎛     d               d     
⎢(ψ - ξ)⋅⎜- ⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅⎜X

Now we need to inject the points motion in the world, with respect to camera inputs. Their motion with respect to the camera velocity is know

In [8]:
vx, vy, vz, wx, wy, wz = sp.symbols("v_x, v_y, v_z, w_x, w_y, w_z")
vc = sp.Matrix([[vx, vy, vz]]).T
wc = sp.Matrix([[wx, wy, wz]]).T
Pw = sp.Matrix([[X(t), Y(t), Z(t)]]).T
Pw_dot = -vc - wc.cross(Pw)
Pw_dot

⎡-vₓ - w_y⋅Z(t) + w_z⋅Y(t)⎤
⎢                         ⎥
⎢-v_y + wₓ⋅Z(t) - w_z⋅X(t)⎥
⎢                         ⎥
⎣-v_z - wₓ⋅Y(t) + w_y⋅X(t)⎦

In [9]:
dimg_with_uc_X = sdimg.subs({sp.Derivative(X(t), t): Pw_dot[0]})
dimg_with_uc_X

⎡        ⎛⎛     _______________________       ⎞                               
⎢        ⎜⎜    ╱  2       2       2           ⎟                             ⎛ 
⎢(ψ - ξ)⋅⎜⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅(-vₓ - w_y⋅Z(t) + w_z⋅Y(t))⋅⎝X
⎢        ⎝                                                                    
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                ⎛  ⎛     _______________________       ⎞                     
⎢                ⎜  ⎜    ╱  2       2       2           ⎟ ⎛                   
⎢        (ψ - ξ)⋅⎜- ⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + 

In [10]:
dimg_with_uc_Y = dimg_with_uc_X.subs({sp.Derivative(Y(t), t): Pw_dot[1]})
dimg_with_uc_Y

⎡        ⎛⎛     _______________________       ⎞                               
⎢        ⎜⎜    ╱  2       2       2           ⎟                             ⎛ 
⎢(ψ - ξ)⋅⎜⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅(-vₓ - w_y⋅Z(t) + w_z⋅Y(t))⋅⎝X
⎢        ⎝                                                                    
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢        ⎛⎛     _______________________       ⎞                               
⎢        ⎜⎜    ╱  2       2       2           ⎟                             ⎛ 
⎢(ψ - ξ)⋅⎜⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅(-v_

In [11]:
dimg_with_uc_Z = dimg_with_uc_Y.subs({sp.Derivative(Z(t), t): Pw_dot[2]})
dimg_with_uc_Z

⎡        ⎛⎛     _______________________       ⎞                               
⎢        ⎜⎜    ╱  2       2       2           ⎟                             ⎛ 
⎢(ψ - ξ)⋅⎝⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅(-vₓ - w_y⋅Z(t) + w_z⋅Y(t))⋅⎝X
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢        ⎛⎛     _______________________       ⎞                               
⎢        ⎜⎜    ╱  2       2       2           ⎟                             ⎛ 
⎢(ψ - ξ)⋅⎝⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅(-v_y + wₓ⋅Z(t) - w_z⋅X(t))⋅⎝X
⎢───────────────────────────────────────────────────

In [12]:
sdimg_uc = sp.simplify(dimg_with_uc_Z)
sdimg_uc

⎡         ⎛⎛     _______________________       ⎞                              
⎢         ⎜⎜    ╱  2       2       2           ⎟                            ⎛ 
⎢-(ψ - ξ)⋅⎝⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅(vₓ + w_y⋅Z(t) - w_z⋅Y(t))⋅⎝X
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢         ⎛⎛     _______________________       ⎞                              
⎢         ⎜⎜    ╱  2       2       2           ⎟                            ⎛ 
⎢-(ψ - ξ)⋅⎝⎝ξ⋅╲╱  X (t) + Y (t) + Z (t)  + Z(t)⎠⋅(v_y - wₓ⋅Z(t) + w_z⋅X(t))⋅⎝X
⎢───────────────────────────────────────────────────

Finally we can take the partial derivatives wrt to each of the camera inputs to create the interaction matrix

In [13]:
L11 = sp.simplify(sdimg_uc[0].diff(vx))
L12 = sp.simplify(sdimg_uc[0].diff(vy))
L13 = sp.simplify(sdimg_uc[0].diff(vz))
L14 = sp.simplify(sdimg_uc[0].diff(wx))
L15 = sp.simplify(sdimg_uc[0].diff(wy))
L16 = sp.simplify(sdimg_uc[0].diff(wz))
L21 = sp.simplify(sdimg_uc[1].diff(vx))
L22 = sp.simplify(sdimg_uc[1].diff(vy))
L23 = sp.simplify(sdimg_uc[1].diff(vz))
L24 = sp.simplify(sdimg_uc[1].diff(wx))
L25 = sp.simplify(sdimg_uc[1].diff(wy))
L26 = sp.simplify(sdimg_uc[1].diff(wz))

L = sp.Matrix([[L11, L12, L13, L14, L15, L16],
                [L21, L22, L23, L24, L25, L26]])
# L = sp.simplify(L)
# L

Once we obtain the final expression, we should be able to obtain the perspective camera interaction matrix by setting the general variables to their corresponding values.

In [14]:
if MODEL == "generalized" and (GENERALIZED == "caron" or GENERALIZED == "barreto"):
    # Substitute for the auxiliary variables
    L = L.subs({rho: r})
    r2 = X(t)**2 + Y(t)**2 + Z(t)**2
    L = L.subs({r2: r**2})

sp.simplify(L)

⎡         ⎛ 2                              2       2        ⎞                 
⎢-(ψ - ξ)⋅⎝ρ ⋅(ρ⋅ξ + Z(t)) - (ρ⋅ξ + Z(t))⋅X (t) + X (t)⋅Z(t)⎠                 
⎢─────────────────────────────────────────────────────────────                
⎢                        2             2                                      
⎢                       ρ ⋅(ρ⋅ξ + Z(t))                                       
⎢                                                                             
⎢                                                                        ⎛ 2  
⎢                     ξ⋅(ψ - ξ)⋅X(t)⋅Y(t)                       -(ψ - ξ)⋅⎝ρ ⋅(
⎢                     ───────────────────                       ──────────────
⎢                                     2                                       
⎣                       ρ⋅(ρ⋅ξ + Z(t))                                        

                                                         ⎛                    
       ξ⋅(ψ - ξ)⋅X(t)⋅Y(t)                       (ψ

In [15]:
if MODEL == "generalized" and (GENERALIZED == "caron" or GENERALIZED == "barreto"):
    # Obtain symbolic model of our matrix for perspective case
    L_p = L.subs({csi: 0, psi: 1})
    barreto_L_p = barreto_L.subs({csi: 0, psi: 1})
L_p

⎡            ⎛ 2       2       2   ⎞                       ⎛ 2       2   ⎞    
⎢-1          ⎝X (t) + Y (t) + Z (t)⎠⋅X(t)    X(t)⋅Y(t)    -⎝X (t) + Z (t)⎠    
⎢────   0    ────────────────────────────    ─────────    ─────────────────   
⎢Z(t)                   2  2                    2                2            
⎢                      ρ ⋅Z (t)                Z (t)            Z (t)         
⎢                                                                             
⎢            ⎛ 2       2       2   ⎞        2       2                         
⎢      -1    ⎝X (t) + Y (t) + Z (t)⎠⋅Y(t)  Y (t) + Z (t)     -X(t)⋅Y(t)      -
⎢ 0    ────  ────────────────────────────  ─────────────     ───────────     ─
⎢      Z(t)             2  2                    2                2            
⎣                      ρ ⋅Z (t)                Z (t)            Z (t)         

     ⎤
Y(t) ⎥
──── ⎥
Z(t) ⎥
     ⎥
     ⎥
     ⎥
X(t) ⎥
─────⎥
Z(t) ⎥
     ⎦

In [16]:
sp.simplify(barreto_L_p)

⎡      _______________________                                        ________
⎢     ╱  2       2       2                                           ╱  2     
⎢    ╱  X (t) + Y (t) + Z (t)                                       ╱  X (t) +
⎢   ╱   ─────────────────────                                  -   ╱   ───────
⎢  ╱             2                                                ╱           
⎢╲╱             Z (t)                                           ╲╱            
⎢─────────────────────────────                0                ───────────────
⎢     _______________________                                       __________
⎢    ╱  2       2       2                                          ╱  2       
⎢  ╲╱  X (t) + Y (t) + Z (t)                                     ╲╱  X (t) + Y
⎢                                                                             
⎢                                     _______________________         ________
⎢                                    ╱  2       2   

In [17]:
if MODEL == "generalized" and True:
    point = [0.5, 0.25, -0.5]
    from math import sqrt
    distance = sqrt(point[0]**2 + point[1]**2 + point[2]**2)
    L_numeric = L.subs({X(t): point[0], Y(t): point[1], Z(t): point[2], r: distance, csi: 0, psi: 1})

In [18]:
L_numeric

⎡2.0   0   2.0  0.5   -2.0  -0.5⎤
⎢                               ⎥
⎣ 0   2.0  1.0  1.25  -0.5  1.0 ⎦

In [19]:
barreto_L_numeric

⎡2.0   0   2.0   -0.5  2.0   0.5⎤
⎢                               ⎥
⎣ 0   2.0  -1.0  1.25  -0.5  1.0⎦

In [20]:
if MODEL == "generalized" and (GENERALIZED == "caron" or GENERALIZED == "barreto"):
    # Then we can simplify further
    L_err = L - barreto_L

    # Numerical substituition with a point
    L_err = L_err.subs({X(t): point[0]})
    L_err = L_err.subs({Y(t): point[1]})
    L_err = L_err.subs({Z(t): point[2]})
    L_err = L_err.subs({r: distance})
    L_err = L_err.subs({csi: 0})
    L_err = L_err.subs({psi: 1})
L_err

⎡4.44089209850063e-16           0            4.44089209850063e-16          1.0
⎢                                                                             
⎣         0            4.44089209850063e-16          2.0           2.220446049

           -4.0  -1.0⎤
                     ⎥
25031e-16   0     0  ⎦

In [21]:
if MODEL == "perspective" and False:
    # substitute expressions and obtain back the perspective interaction matrix
    L = L.subs({r11: 1})
    L = L.subs({r33: 1})
    L = L.subs({csi: 0})
    L = L.subs({psi: 1})
    L = L.subs({mc: 1})

    # These next substitutions are not doing anything, don't know why:
    x2 = X(t)**2 / (Z(t)**2)
    y2 = Y(t)**2 / (Z(t)**2)

    L = L.subs({x2: x**2})
    L = L.subs({y2: y**2})

    L = L.subs({X(t)/Z(t): x})
    L = L.subs({Y(t)/Z(t): y})

    # But the output is correct, corresponding to the interaction matrix of a perspective camera
    sp.simplify(L)

Check!